In [115]:
#импорт необходимых библиотек и модулей

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

# Домашнее задание 4.
# Коллаборативная фильтрация - продолжение. Поиск похожих товаров и пользователей

## Задание 1. Перенести метрики из ДЗ-1 в metrics.py

In [116]:
#проверка корректности импорта модуля
from metrics import precision_at_k, recall_at_k

## Задание 2. Перенеcти функцию prefilter_items в utils.py

In [117]:
#проверка корректности импорта модуля
from utils import prefilter_items

Проверяем работу функции предобработки prefilter_items

In [118]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [119]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [120]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [121]:
# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.shape, data_test.shape

((1296382, 12), (40054, 12))

In [122]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[856942, 865456, 951954, 958046, 969231, 97158..."
1,3,"[835476, 872021, 909638, 1083328, 1096727]"


In [123]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818981,819063,819112,819304,819330,819400,819927,819978,820082,820291,...,15827180,15831209,15831255,15926712,15926775,15926844,15926885,15926886,15926887,15926927
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [125]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [126]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)]
    return res

In [127]:
model_bpr = BayesianPersonalizedRanking(factors=200, 
                                regularization=0.01,
                                learning_rate=0.01,
                                iterations=25,  
                                num_threads=4)

model_bpr.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

In [128]:
%%time

model = AlternatingLeastSquares(factors=200, 
                                regularization=0.01,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 2.56 s


In [129]:
result['bm25_bpr'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_bpr, N=5))
result['bm25_als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.head(2)

,user_id,actual,bm25_bpr,bm25_als
0,1,"[856942, 865456, 951954, 958046, 969231, 97158...","[995965, 1027569, 862349, 7024990, 912704]","[1082212, 1096317, 878285, 848761, 844818]"
1,3,"[835476, 872021, 909638, 1083328, 1096727]","[1118533, 1029624, 871756, 893018, 859075]","[877358, 993028, 9337369, 1020581, 998206]"


In [130]:
result.apply(lambda row: precision_at_k(row['bm25_bpr'], row['actual']), axis=1).mean()

0.048232189973615065

In [131]:
result.apply(lambda row: precision_at_k(row['bm25_als'], row['actual']), axis=1).mean()

0.10955145118733463

**Вывод:** лучший результат дает bm25_als.

## 3. Создайте модуль recommenders.py. Напишите код для класса MainRecommender

In [132]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
                
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = \
        self.prepare_dicts(self.user_item_matrix)
        
        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        #self.item_id_to_ctm = #your_code
        
        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        """Подготавливает user_item матрицу"""
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=200, regularization=0.01, iterations=20, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        # находим top-N покупок user
        top = self.top_purchases[self.top_purchases['user_id'] == user].head(N).item_id.tolist()
        
        # для каждого из top-N покупок user находим 1 наиболее ближайший товар и делаем рекомендации
        recs = []
        for item_id in top:
            rec = [self.id_to_itemid[i[0]] for i in self.model.similar_items(self.itemid_to_id[item_id], N = 3)]
            rec_id = [i for i in rec if i != 999999] #отсеиваем 999999
            semilar_item = rec_id[1] #отсеиваем сам товар
            recs.append(semilar_item)

        #только уникальные рекоммендации
        recs = list(set(recs))
        
        return recs
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        """top-1 от каждого из N-ти похожих юзеров"""
        #находим N наиболее похожих юзеров
        users_id = [self.id_to_userid[i[0]] for i in self.model.similar_users(self.userid_to_id[user], N=N+1)] 
        similar_users = [i for i in users_id  if i != user] 
        
        #N покупок по 1 самой популярной от каждого из N ближайших юзеров 
        top_purchases_n_users = self.top_purchases.groupby('user_id').head(1)
        top_purchases_n_users = top_purchases_n_users[top_purchases_n_users['user_id'].isin(similar_users)]
        top_purchases_n_users.sort_values('user_id', ascending=False, inplace=True)
        recs = top_purchases_n_users.item_id.tolist()
        
        return recs  
    
    def get_similar_users_recommendation_2(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        """top-N среди всех покупок N-ти похожих юзеров"""
        #находим N наиболее похожих юзеров
        users_id = [self.id_to_userid[i[0]] for i in self.model.similar_users(self.userid_to_id[user], N=N+1)] 
        similar_users = [i for i in users_id  if i != user] 
        
        #товары, купленные N-похожими юзерами
        top_purchases_n_users = self.top_purchases[self.top_purchases['user_id'].isin(similar_users)]
        
        #top-N товаров, купленных N-похожими юзерами
        recs = top_purchases_n_users.head(N).item_id.tolist()
                              
        #только уникальные рекоммендации
        recs = list(set(recs))
        
        return recs
    
    def get_similar_users_recommendation_3(self, user, N=5):
        """Рекомендуем топ-N товаров, которые нравятся похожим пользователям"""
        #находим N наиболее похожих юзеров
        users_id = [self.id_to_userid[i[0]] for i in self.model.similar_users(self.userid_to_id[user], N=15)] 
        similar_users = [i for i in users_id  if i != user] 
        
        #для каждого ближайших N юзеров находим 1 наиболее ближайший товар и делаем рекомендации
        recs = []
        for similar_user in similar_users:
            rec = [self.id_to_itemid[i[0]] for i in 
                    self.own_recommender.recommend(userid=self.userid_to_id[similar_user], 
                                    user_items=csr_matrix(self.user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=1, 
                                    filter_already_liked_items=False, 
                                    filter_items=[self.itemid_to_id[999999]],  # !!! 
                                    recalculate_user=False)]
            recs.append(rec[0])
            
        #только уникальные рекоммендации
        recs = list(set(recs))
        
        return recs

In [133]:
#проверка корректности импорта модуля
from recommenders import MainRecommender

In [134]:
#data_train уже предобработана
recommender = MainRecommender(data_train)

result['similar_items_rec'] = result['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=5))
result['similar_users_rec'] = result['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=5))
result['similar_users_rec_2'] = result['user_id'].apply(lambda x: recommender.get_similar_users_recommendation_2(x, N=5))
result.head(2)

,user_id,actual,bm25_bpr,bm25_als,similar_items_rec,similar_users_rec,similar_users_rec_2
0,1,"[856942, 865456, 951954, 958046, 969231, 97158...","[995965, 1027569, 862349, 7024990, 912704]","[1082212, 1096317, 878285, 848761, 844818]","[9526410, 9297615, 7135183, 1011089, 5577022]","[1055425, 885290, 865456, 854133, 898847]","[862349, 852014, 898847, 1082212, 854133]"
1,3,"[835476, 872021, 909638, 1083328, 1096727]","[1118533, 1029624, 871756, 893018, 859075]","[877358, 993028, 9337369, 1020581, 998206]","[864774, 901543, 1010440, 1075979, 909811]","[1029624, 907662, 1072917, 10149656, 937791]","[962229, 962229, 907662, 1072917, 901762]"


In [135]:
similar_items_rec_metric = result.apply(lambda row: precision_at_k(row['similar_items_rec'], row['actual']), axis=1).mean()
print(f'Метрика рекомендаций товаров, похожих на топ-5 купленных юзером товаров: {similar_items_rec_metric}')

Метрика рекомендаций товаров, похожих на топ-5 купленных юзером товаров: 0.04460007047216373


C:\Users\BazhanovaEN\Recommendation_systems\metrics.py:30: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


In [136]:
similar_users_rec_metric = result.apply(lambda row: precision_at_k(row['similar_users_rec'], row['actual']), axis=1).mean()
print(f'Метрика рекомендаций top-5 товаров, среди купленных 5-тью похожими юзерами (top-1 от каждого из 5-ти похожих юзеров):{similar_users_rec_metric}')

Метрика рекомендаций top-5 товаров, среди купленных 5-тью похожими юзерами (top-1 от каждого из 5-ти похожих юзеров):0.028009871320289165


In [137]:
similar_users_rec_metric_2 = result.apply(lambda row: precision_at_k(row['similar_users_rec_2'], row['actual']), axis=1).mean()
print(f'Метрика рекомендаций top-5 товаров, среди купленных 5-тью похожими юзерами (top-5 среди всех покупок 5-ти похожих юзеров):{similar_users_rec_metric_2}')

Метрика рекомендаций top-5 товаров, среди купленных 5-тью похожими юзерами (top-5 среди всех покупок 5-ти похожих юзеров):0.03225806451612914


In [138]:
metrics_result = pd.DataFrame({'ALS_similar_items_rec': [similar_items_rec_metric], 'ALS_similar_users_rec': [similar_users_rec_metric],'ALS_similar_users_rec_2': [similar_users_rec_metric_2]}, index = {'precision@5'})
metrics_result

,ALS_similar_items_rec,ALS_similar_users_rec,ALS_similar_users_rec_2
precision@5,0.0446,0.02801,0.032258


**Вывод:** лучшие рекомендации: товары, похожие на топ-N купленных юзером